In [1]:
### Import packages ###
import itertools
import pandas as pd

# Input

In [2]:
AbbreviationDictionary = {"BankNote": "BN",
                          "Bar7": "B7",
                          "BreastCancer": "BC",
                          "CarEvaluation": "CE",
                          "COMPAS": "CP",
                          "FICO": "FI",
                          "Haberman": "HM",
                          "Iris": "IS",
                          "MONK1": "M1",
                          "MONK3":"M3"}

In [3]:
BatchSize = 5
DiversityWeight = 0.4

In [4]:
# Input Data Set #
Data = "CarEvaluation" # Iris  MONK1  MONK3  Bar7 (10)  COMPAS (50) | BankNote (10)  BreastCancer (5)  CarEvaluation (5)  FICO (50)  Haberman 
JobNameAbbrev = AbbreviationDictionary[Data]

# Input directory #
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Parameters #
ParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,50)),
                      #  "Seed":list([]),                       
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["BatchQBCDiversityFunction"],
                       "ModelType":["TreeFarmsFunction"],
                       "UniqueErrorsInput": [0,1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                         # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0.005],
                       "Type": ["Classification"],
                       "DiversityWeight": [DiversityWeight],
                       "BatchSize": [BatchSize],
                       "Partition": ["medium"],                                                        # [short, medium, long, largemem, compute, cpu-g2-mem2x]
                       "Time": ["2-23:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": ["30000M"]}                                                                # [100M, 30000M, 100000M]

# Create Parameter Vector #
ParameterVector = pd.DataFrame.from_records(itertools.product(*ParameterDictionary.values()), columns=ParameterDictionary.keys())

# Include/exclude Passive Learning

In [5]:
### Include Random Forest ###
RandomForestParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,50)),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["PassiveLearning"],
                       "ModelType":["RandomForestClassificationFunction"],
                       "UniqueErrorsInput": [1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                                    # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0],
                       "Type": ["Classification"],
                       "DiversityWeight": [0],
                       "BatchSize": [BatchSize],
                       "Partition": ["short"],                                                        # [short, medium, long, largemem, or compute]
                       "Time": ["00:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": [1000]}                                                                # [1000, 30000, 100000]

RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# NOTE: Comment out chunk to not include random forest simulations. ###
ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
ParameterVector = ParameterVector.sort_values("Seed")
ParameterVector.index = range(0, ParameterVector.shape[0])

# Include/exclude Random Forest QBC Simulations

In [6]:
### Include Random Forest ###
RandomForestParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,50)),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["BatchQBCDiversityFunction"],
                       "ModelType":["RandomForestClassificationFunction"],
                       "UniqueErrorsInput": [0],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                                    # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0],
                       "Type": ["Classification"],
                       "DiversityWeight": [DiversityWeight],
                       "BatchSize": [BatchSize],
                       "Partition": ["short"],                                                        # [short, medium, long, largemem, or compute]
                       "Time": ["00:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": [1000]}                                                                # [1000, 30000, 100000]

RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# NOTE: Comment out chunk to not include random forest simulations. ###
ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
ParameterVector = ParameterVector.sort_values("Seed")
ParameterVector.index = range(0, ParameterVector.shape[0])

# Job and Output Name

In [7]:
### Remove Dictionary ###
del ParameterDictionary

# Generate JobName #
ParameterVector["JobName"] = (
    ParameterVector["Seed"].astype(str) +
    JobNameAbbrev + 
    "_MT" + ParameterVector["ModelType"].astype(str) +
    "_UEI" + ParameterVector["UniqueErrorsInput"].astype(str) +
    "_" + ParameterVector["RashomonThresholdType"].astype(str) + 
    ParameterVector["RashomonThreshold"].astype(str)+
    "_D" + ParameterVector["DiversityWeight"].astype(str) + 
    "B" + ParameterVector["BatchSize"].astype(str))

# Replace Job Name #
ParameterVector["JobName"] = (
    ParameterVector["JobName"]
    .str.replace(r"_MTTreeFarmsFunction_UEI0_", "_D", regex=True)
    .str.replace(r"_MTTreeFarmsFunction_UEI1_", "_U", regex=True)
    .str.replace(r"Adder", "A", regex=True)
    .str.replace(r"Multiplier", "M", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI0_", "_RF", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI1_", "_PL", regex=True)
    .str.replace(r"0.", "", regex=False)
    )

# Output Name #
ParameterVector["Output"] = ParameterVector["Data"].astype(str) + "/" + ParameterVector["ModelType"].astype(str) + "/Raw/" + ParameterVector["JobName"] + ".pkl"
ParameterVector["Output"] = ParameterVector["Output"].str.replace("Function", "", regex=False)

---

# Only run these ones

In [8]:
# ### Simulations that failed ###
FilterText = ['2CE_DA005_D4B5', '3CE_DA005_D4B5', '11CE_DA005_D4B5', '15CE_DA005_D4B5', '19CE_DA005_D4B5', '31CE_DA005_D4B5', '32CE_DA005_D4B5', '33CE_DA005_D4B5', '34CE_DA005_D4B5', '37CE_DA005_D4B5', '38CE_DA005_D4B5', '42CE_DA005_D4B5', '43CE_DA005_D4B5', '44CE_DA005_D4B5', '45CE_DA005_D4B5', '46CE_DA005_D4B5', '47CE_DA005_D4B5', '48CE_DA005_D4B5', '12CE_UA005_D4B5', '13CE_UA005_D4B5', '16CE_UA005_D4B5', '19CE_UA005_D4B5', '23CE_UA005_D4B5', '32CE_UA005_D4B5', '33CE_UA005_D4B5', '38CE_UA005_D4B5', '42CE_UA005_D4B5', '43CE_UA005_D4B5', '44CE_UA005_D4B5', '45CE_UA005_D4B5', '48CE_UA005_D4B5']

### Rerun these simulations ###
ParameterVector = ParameterVector[ParameterVector["JobName"].isin(FilterText)]
ParameterVector.index = range(0,len(ParameterVector))
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,RashomonThresholdType,RashomonThreshold,Type,DiversityWeight,BatchSize,Partition,Time,Memory,JobName,Output
0,CarEvaluation,2,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,0,100,0.01,Adder,0.005,Classification,0.4,5,medium,2-23:59:00,30000M,2CE_DA005_D4B5,CarEvaluation/TreeFarms/Raw/2CE_DA005_D4B5.pkl
1,CarEvaluation,3,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,0,100,0.01,Adder,0.005,Classification,0.4,5,medium,2-23:59:00,30000M,3CE_DA005_D4B5,CarEvaluation/TreeFarms/Raw/3CE_DA005_D4B5.pkl
2,CarEvaluation,11,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,0,100,0.01,Adder,0.005,Classification,0.4,5,medium,2-23:59:00,30000M,11CE_DA005_D4B5,CarEvaluation/TreeFarms/Raw/11CE_DA005_D4B5.pkl
3,CarEvaluation,12,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.005,Classification,0.4,5,medium,2-23:59:00,30000M,12CE_UA005_D4B5,CarEvaluation/TreeFarms/Raw/12CE_UA005_D4B5.pkl
4,CarEvaluation,13,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.005,Classification,0.4,5,medium,2-23:59:00,30000M,13CE_UA005_D4B5,CarEvaluation/TreeFarms/Raw/13CE_UA005_D4B5.pkl
5,CarEvaluation,15,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,0,100,0.01,Adder,0.005,Classification,0.4,5,medium,2-23:59:00,30000M,15CE_DA005_D4B5,CarEvaluation/TreeFarms/Raw/15CE_DA005_D4B5.pkl
6,CarEvaluation,16,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.005,Classification,0.4,5,medium,2-23:59:00,30000M,16CE_UA005_D4B5,CarEvaluation/TreeFarms/Raw/16CE_UA005_D4B5.pkl
7,CarEvaluation,19,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.005,Classification,0.4,5,medium,2-23:59:00,30000M,19CE_UA005_D4B5,CarEvaluation/TreeFarms/Raw/19CE_UA005_D4B5.pkl
8,CarEvaluation,19,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,0,100,0.01,Adder,0.005,Classification,0.4,5,medium,2-23:59:00,30000M,19CE_DA005_D4B5,CarEvaluation/TreeFarms/Raw/19CE_DA005_D4B5.pkl
9,CarEvaluation,23,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.005,Classification,0.4,5,medium,2-23:59:00,30000M,23CE_UA005_D4B5,CarEvaluation/TreeFarms/Raw/23CE_UA005_D4B5.pkl


---

# Save Parameter Vector

In [9]:
### Resave ###
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)